# Предсказание цены на автомобили

Датасет содержит 10 000 записей, созданных с целью прогнозирования цен на автомобили. Каждая строка представляет информацию об автомобиле и его цене.

**Описание столбцов:**

- *Brand*: марка автомобиля.

- *Model*: модель автомобиля.

- *Year*: год выпуска.

- *Engine_Size*: размер двигателя в литрах.

- *Fuel_Type*: тип топлива, используемого автомобилем:
    - *Petrol*: автомобили, работающие на бензине.
    - *Diesel*: автомобили, работающие на дизельном топливе.
    - *Hybrid*: автомобили, использующие как топливо, так и электричество.
    - *Electric*: полностью электрические автомобили.

- *Transmission*: тип трансмиссии:
    - *Manual*: механическая коробка передач.
    - *Automatic*: автоматическая коробка передач.
    - *Semi-Automatic*: полуавтоматическая коробка передач.

- *Mileage*: общее расстояние (в км), которое проехал автомобиль.

- *Doors*: количество дверей в автомобиле.

- *Owner_Count*: количество предыдущих владельцев автомобиля.

- *Price*: цена автомобиля (таргет).

In [ ]:
import optuna as opt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

sns.set_theme()

## Знакомство с данными

In [ ]:
data = pd.read_csv('../data/car_price_dataset.csv')
data

In [ ]:
# проверим данные на наличие пропусков
data.info()

In [ ]:
# проверим на наличие выбросов числовые признаки
data.describe()

## EDA

In [ ]:
# взглянем на статистику категориальных данных
data.describe(include=['object'])

In [ ]:
# взглянем, как модель влияет на ценообразование
price_by_model = data.groupby('Model')['Price'].mean().sort_values(ascending=False)

plt.figure(figsize=(15, 6))
sns.barplot(price_by_model)
plt.xticks(rotation=90)
plt.show()
print('Среднеквадратическое отклонение:', price_by_model.std())

## Подготовка данных

In [23]:
# закодируем порядковые признаки
ord_encoder = LabelEncoder()

data['Doors'] = ord_encoder.fit_transform(data['Doors'])
data['Owner_Count'] = ord_encoder.fit_transform(data['Owner_Count'])
data['Year'] = ord_encoder.fit_transform(data['Year'])

In [24]:
# закодируем номинальные признаки one hot encoding
columns_encode = ['Brand', 'Model', 'Fuel_Type', 'Transmission']

data = pd.get_dummies(data, columns=columns_encode, drop_first=True, dtype='int')

In [25]:
# подготовим тренировочную и тестовую выборки
X = data.drop(['Price'], axis=1)
y = data['Price']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [26]:
# нормализуем числовые признаки с помощию minmaxscaler
num_columns = ['Engine_Size', 'Mileage']

scaler = MinMaxScaler()
X_train[num_columns] = scaler.fit_transform(X_train[num_columns])
X_test[num_columns] = scaler.transform(X_test[num_columns])

## Обучение модели

In [ ]:
# в качестве модели регрессии используем RandomForest
forest = RandomForestRegressor(
    n_estimators=100,
    criterion='squared_error',
    min_samples_leaf=5,
    max_depth=10,
    random_state=42
)

forest.fit(X_train, y_train)

## Оценка модели

In [ ]:
print('MAE:', mean_absolute_error(y_test, forest.predict(X_test)))
print('R^2:', r2_score(y_test, forest.predict(X_test)))

## Подбор гиперпараметров

In [ ]:
# подберем оптимальные гиперпараметры для нашей модели с помощью optuna и кросс-валидации
def opt_forest(trial):
    criterion = trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'])
    max_depth = trial.suggest_int('max_depth', 10, 20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, step=1)
    
    model = RandomForestRegressor(
        n_estimators=100,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        n_jobs=-1,
        random_state=42
    )
    
    score = cross_val_score(
        model, X_train, y_train,
        scoring='r2',
        cv=5,
        n_jobs=-1
    ).mean()
    
    return score

In [ ]:
forest_study = opt.create_study(study_name='RandomForest', direction='maximize')
forest_study.optimize(opt_forest, n_trials=10, n_jobs=-1)

In [ ]:
print('Best params:', forest_study.best_params)

In [ ]:
forest = RandomForestRegressor(
    **forest_study.best_params,
    n_estimators=100,
    random_state=42
)

forest.fit(X_train, y_train)

print('MAE:', mean_absolute_error(y_test, forest.predict(X_test)))
print('R^2:', r2_score(y_test, forest.predict(X_test)))